# Adding Pickup Locations, Drones, and Using BART


## Context

Adding more pickup locations may help to grow the customer base and increase the frequency at which customers purchase meals. Locations near BART stations would be good choices because riders could easily pick up meals at or near the stations they travel through on the way to or from work. We also would prefer to pilot low-cost solutions and avoid costly real estate and long-term contracts.

Additionally, combining new pickup locations with delivery drones is an intriguing option because it drastically expands the delivery range for AGM in the Bay Area. By strategically selecting where to base the drone delivery from, AGM delivery can be expanded to cover the most populous parts of the Bay Area.

Choosing to establish pick-up locations near or at BART will also allow us to utilize this underground transit system in order to move supply. In this area, dense traffic and a bay make delivery by car less viable. Rising costs of gas make car-delivery an even less appealing option. Utilizing specialized backpacks such as those used by successful food delivery apps, our workers should be able to easily transport sufficient product in order to have reliable supplies at our pick-up locations. 

In choosing potential Bart Stations for our pick-up locations, we will consider:

- The betweeness centrality of the each BART Station at the new pick-up locations. This indicates how many paths pass through that station. 
- The travel time via BART from our Berkeley Store's local BART station to each new pick-up location. 
- The local population density. We will analyze population within a 1.5 mile radius from each BART station.
- The distance between each of the chosen pick-up locations. 

## Methodology

We will examine each station's betweenness centrality, travel time from the Ashby station, and surrounding population (within a 1.5 mile radius, the delivery range for a drone). Betweenness centrality will indicate the number of routes which pass through that station. Stations with dense surrounding populations and high betweenness centrality are likely good candidates for pickup locations because this would not only enable us to capture customers with drone deliveries based from those stations, but also commuters who enter, exit, or otherwise pass through those stations.

We will start by identifying a list of stations which higher betweeness centrality compared to the Berkeley store and evaluate based on population density within a 1.5 mile radius. Next, we will refine the list of potential pickup locations to minimize travel time between the stations and the Ashby station, as meals will be prepared at the Berkeley store and transported via BART to the pickup locations. Finally, we will identify the potential market share we could capture with an additional 1.5 mile reach using a drone delivery option (where the drones are based out of the pickup location). We will select 4 BART stations from this final list within which to establish pickup locations.

# Included Modules and Packages

In [161]:
import neo4j

import csv
import json

import math
import numpy as np
import pandas as pd

import psycopg2
from geographiclib.geodesic import Geodesic

import warnings
warnings.filterwarnings("ignore")

import gmaps
import gmaps.geojson_geometries

# Supporting Code

In [162]:
# Connect to Neo4j

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))
session = driver.session(database="neo4j")

# Connect to PostgreSQL

connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

cursor = connection.cursor()

In [163]:
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [164]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [165]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    
    for row in rows:
        zip, population = row[0], row[1]
        total_population += population
    return float(total_population)  

In [166]:
def my_station_get_zip_list(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    
    zip_list = []
    
    for row in rows:
        zip = row[0]
        population = row[1]
        total_population += population
        zip_list.append(row[0])
    return zip_list

In [167]:
def cleanse_stations(df):
    """Returns a data frame with unique station names cleansed of line(s) and depart, arrive"""
    
    words = ["blue", "green", "orange", "red", "yellow", "orange", "gray", "depart", "arrive"]
    regex_pattern = r'\b(?:{})\b'.format('|'.join(words))
    df["name"] = df["name"].str.replace(regex_pattern, '')
    return df

In [168]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [169]:
## Import Google Maps API key

f = open('../gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

# Generate Data Frame for Analysis

In [170]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station,
        latitude,
        longitude
from stations
order by station

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

##### Add population within 1.5 miles of each station, which is the delivery range for a drone

In [171]:
df["pop_1_5"] = [my_station_get_zips(station, 1.5) for station in df["station"]]

##### Add betweenness centrality, which measures the number of paths which pass through a node (station). High betweenness centrality for a station indicates a high number of paths which pass through that station.

In [172]:
# Betweenness centrality

query = """

CALL gds.betweenness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

bet_df = my_neo4j_run_query_pandas(query)

# Remove the line and depart / arrive designations

bet_df = cleanse_stations(bet_df)

# Keep the entry for each station with the maximum betweenness centrality

bet_df = bet_df.groupby(["name"])["betweenness"].max()
bet_df = bet_df.to_frame()

# Add degree centrality to df

df["bet_centrality"] = bet_df["betweenness"].values

# Set the index

df.set_index("station", inplace=True)

##### Impute population values for Antioch, Milpitas, OAK, and Pittsburg. These records are missing population values because the lat/lon point location of their corresponding zip codes falls outside of the 1.5 mile radius.

In [173]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from zip_codes

"""

temp = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [174]:
# Using the zip_codes table, find the population for each of the four corresponding zip codes

antioch_station_zip = "94509"
milpitas_station_zip = "95035"
OAK_station_zip = "94621"
pittsburg_station_zip = "94565"

antioch_pop = int(temp.loc[temp["zip"] == antioch_station_zip, "population"])
milpitas_pop = int(temp.loc[temp["zip"] == milpitas_station_zip, "population"])
OAK_pop = int(temp.loc[temp["zip"] == OAK_station_zip, "population"])
pittsburg_pop = int(temp.loc[temp["zip"] == pittsburg_station_zip, "population"])

In [175]:
# Assign the population values back to the data frame

df.loc[df.index=="Antioch", "pop_1_5"] = antioch_pop
df.loc[df.index=="Milpitas", "pop_1_5"] = milpitas_pop
df.loc[df.index=="OAK", "pop_1_5"] = OAK_pop
df.loc[df.index=="Pittsburg", "pop_1_5"] = pittsburg_pop

# Analysis

## Identify which stations look like good candidates for a pickup location

##### Start by finding which stations have a betweenness centrality value that is larger than the Ashby station's and sort by population, keeping the top 15

In [181]:
# Create values for Ashby

berk_pop_1_5 = df.loc[df.index == "Ashby", "pop_1_5"][0]
berk_bet_cent = df.loc[df.index == "Ashby", "bet_centrality"][0]

In [187]:
# Sort based on population

df = df[(df["bet_centrality"] > berk_bet_cent)].sort_values("pop_1_5", ascending=False).head(15)

## Refine the candidate stations based on accessibility to the Ashby Station

##### Travel time between the Ashby BART station and the new pickup locations should be minimized, as food will be prepared at the Berkeley store location and carried to the new pickup locations via public transit.

In [107]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            previous = int(costs[i])
            i += 1
    
    return total_cost

In [113]:
def dist_from_ashby(station):
    
    arrive = "arrive " + str(station)
    
    return my_neo4j_shortest_path("depart Ashby", arrive)

##### Add a column for travel time from the Ashby station

In [184]:
df["stations"] = df.index
df["travel_time_from_ashby"] = df["stations"].apply(dist_from_ashby)
df.drop(["stations"], axis=1, inplace=True)

## Select 4 stations that offer the largest potential new customer market (without consideration of delivery area overlap)

##### A drone delivery option will enable us to capture additional customers within a 1.5 mile radius of pickup locations

We can start our drone placement decision by using previously calculated data to see which stations have the highest population within 1.5 miles. This distance is ideal because it is the maximum delivery range for a drone so we can see the number of potential customers in range. However, we need to consider more than just total population for a potential drone location because we want to spread them out such that drone delivery ranges do not overlap or overlap as little as possible. We want to avoid this overlap because it means we are not maximizing the number of deliverable customers per drone. In the below visual, we can see a naive approach to drone placement that only uses maximum deliverable population per drone rather than minimizing overlap.

In [26]:
sixteenth_st = (37.764847,-122.420042)
glen_park = (37.733118,-122.433808)
civic_center = (37.779861,-122.413498)
powell_streeet = (37.784000,-122.408000)
twenty_fourth_st = (37.752000,-122.418700)
lake_merritt = (37.797773,-122.266588)
fruitvale=(37.774800,-122.224100)
san_leandro=(37.721764,-122.160684)
hayward = (37.669700,-122.087000)

fig = gmaps.figure(center=fruitvale, zoom_level=11)
drawing = gmaps.drawing_layer(features=[
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=sixteenth_st,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=glen_park,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=civic_center,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=powell_streeet,
        stroke_color='red', fill_color=(255, 0, 132)
    )
], mode='DISABLED')
fig.add_layer(drawing)
marker_layer = gmaps.marker_layer([
    sixteenth_st, 
    civic_center, 
    glen_park, 
    powell_streeet
], info_box_content=[
    '16th Street Station',
    'Civic Center Station',
    'Glen Park Station',
    'Powell Street Station'
])
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Now, we consider minimizing delivery area overlap

As we can see above, there is quite a bit of overlap if we only select drone locations by nearby population. In order to help decide more appropriate drone placement locations, we can visualize the population in the Bay Area and use that to determine more optimal locations. The below visual shows the population by zip code in the Bay Area. Darker red shading indicates a higher population in that area.

In [27]:
rollback_before_flag = True
rollback_after_flag = True

# Get population by zip code
query = """
select zip, population, city
from zip_codes
where state = 'CA'
"""

zip_codes = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# Calculate ranges of populations in zip codes
zip_code_quantiles = zip_codes[(zip_codes['city'] == 'San Francisco') | (zip_codes['city'] == 'Oakland')]['population'].quantile([0.2,0.4,0.6,0.8])

berkeley_store = (37.8555, -122.2604)
civic_center = (37.779861,-122.413498)
sixteenth_st=(37.764847,-122.420042)
glen_park = (37.733118,-122.433808)
lake_merritt = (37.797773,-122.266588)
fruitvale=(37.774800,-122.224100)
san_leandro=(37.721764,-122.160684)
hayward = (37.669700,-122.087000)


# Get geojson data for California
f = open('../data/geojson_data/ca_california_geojson.json')
ca_customer_zip_geojson = json.load(f)
f.close()

# Determine the correct color for each zip based on population
# The populations are in the zip_codes table in the DB,
# we need to match each of the zips in the geojson file with the populations in the zip_codes table
colors = [
    (220,220,220) if len(zip_codes[zip_codes['zip'] == zip_code['properties']['ZCTA5CE10']]['population']) == 0 
    else (233,62,58) if zip_codes[zip_codes['zip'] == zip_code['properties']['ZCTA5CE10']]['population'].iloc[0] > zip_code_quantiles.iloc[3]
    else (237,104,60) if zip_codes[zip_codes['zip'] == zip_code['properties']['ZCTA5CE10']]['population'].iloc[0] > zip_code_quantiles.iloc[2]
    else (243,144,63) if zip_codes[zip_codes['zip'] == zip_code['properties']['ZCTA5CE10']]['population'].iloc[0] > zip_code_quantiles.iloc[1]
    else (253,199,12) if zip_codes[zip_codes['zip'] == zip_code['properties']['ZCTA5CE10']]['population'].iloc[0] > zip_code_quantiles.iloc[0]
    else (255,243,59)
    for zip_code in ca_customer_zip_geojson['features']
]

fig = gmaps.figure(center=fruitvale, zoom_level=11)
geojson_layer = gmaps.geojson_layer(ca_customer_zip_geojson, fill_color=colors)

fig.add_layer(geojson_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Updated Drone Locations
In the below chart, we can see more thoughtfully placed delivery drone locations. There is minimal overlap between delivery zones and the zones cover many of the most populated zip codes in the Bay Area. In fact, based on the population within 1.5 miles of each of the stations, there are 671861 customers within the delivery range for the selected drone locations.

In [28]:
fig = gmaps.figure(center=fruitvale, zoom_level=11)
drawing = gmaps.drawing_layer(features=[
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=civic_center,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=glen_park,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=lake_merritt,
        stroke_color='red', fill_color=(255, 0, 132)
    ),
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=fruitvale,
        stroke_color='red', fill_color=(255, 0, 132)
    )
], mode='DISABLED')
fig.add_layer(drawing)
marker_layer = gmaps.marker_layer([
    berkeley_store, 
    civic_center, 
    glen_park, 
    lake_merritt, 
    fruitvale
], info_box_content=[
    'Berkeley AGM Store',
    'Civic Center Station',
    'Glen Park Station',
    'Lake Merritt Station',
    'Fruitvale Station'
])
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
print(
    'Number of customers in delivery range:',
    df[
        (df.index == 'Civic Center') |
        (df.index == 'Glen Park') |
        (df.index == 'Lake Merritt') |
        (df.index == 'Fruitvale')
    ]['pop_1_5'].sum()
)


Number of customers in delivery range: 671861.0


## Costs
While we expect significant benefits from AGM expansion in the Bay Area, there are also costs that need to be taken into account. Much of the expansion has centered around using BART stations for delivery and pickup locations, however each BART ride has a cost and the more extensive use of the BART system, the higher the costs will be. In order to determine how many BART trips are required to fulfill the pickup requests, we decided to extrapolate the number of delivery orders took place in the Peak Deliveries proof-of-concept. In the one-day trial, Peak delivered 540 meals to customers from the Berkeley store. It is difficult to forecast future demand, but we project to double the number of meals delivered by Peak because there are several times more customers in our delivery range, but there may be some uncetainty in our customers using these new options. Our projection is to deliver 1080 meals per day across the different locations with our new delivery options. The final piece of the puzzle is the number of meals that can be transported in a single trip using BART. Based on our industry knowledge, we are projecting a carrier can transport 30 meals in each trip, which would require 9 round-trip BART rides for each location.

### Costs by Station
BART has a variable cost based on entry and exit station. Because of this, we need to determine the cost for trips from the nearest Berkeley store station to each of the new pickup locations. Below we can see the daily costs to get the meals to each station.

In [30]:
number_of_trips = 9
glen_park_costs = round(9.10 * number_of_trips, 2)
civic_center_costs = round(8.30 * number_of_trips, 2)
lake_merritt_costs = round(4.20 * number_of_trips, 2)
fruitvale_costs = round(4.60 * number_of_trips, 2)
daily_transportation_costs = glen_park_costs + civic_center_costs + lake_merritt_costs + fruitvale_costs

print('Glen Park transportation costs: $' + "{:.2f}".format(glen_park_costs))
print('Civic Center transportation costs: $' + "{:.2f}".format(civic_center_costs))
print('Lake Merritt transportation costs: $' + "{:.2f}".format(lake_merritt_costs))
print('Fruitvale transportation costs: $' + "{:.2f}".format(fruitvale_costs))
print('Total daily transportation costs: $' + "{:.2f}".format(daily_transportation_costs))

Glen Park transportation costs: $81.90
Civic Center transportation costs: $74.70
Lake Merritt transportation costs: $37.80
Fruitvale transportation costs: $41.40
Total daily transportation costs: $235.80


### Drone Price
Based on preliminary research, we have found the retail price for a delivery drone is \\$4,000. In Phase 1 of the new delivery options, we recommend starting with one drone per pickup location which would total at \$16,000.

## Projected Profits
Based on our projections of 1080 meals sold in a day, that would amount to \\$12,960 in new daily sales based on the fixed price of \$12 per meal. Below we can see the projected overall annual profits from the new delivery/pickup options.

In [31]:
meal_price = 12
projected_daily_meals_sold = 1080
projected_daily_income = projected_daily_meals_sold * meal_price
initial_drone_cost = 16000
annual_business_days = 363

annual_profit = ((projected_daily_income - daily_transportation_costs) * annual_business_days) - initial_drone_cost
print('Projected annual profit: $' + "{:.2f}".format(annual_profit))

Projected annual profit: $4602884.60


In [190]:
projected_daily_income * 363

4704480